In [13]:
import os
import pandas as pd
from newsapi.newsapi_client import NewsApiClient
import nltk
from nltk.corpus import reuters
from datetime import date, datetime, timedelta
import pickle
%matplotlib inline
news_api = NewsApiClient(api_key='8bd0a8ca889b4a2e9626db863dba01ca')

In [14]:

nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/songzhao/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# News Headlines Sentiment

Use the news api to pull the latest news articles for bitcoin and ethereum and create a DataFrame of sentiment scores for each coin. 

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [16]:
# Read your api key environment variable
api_key='8bd0a8ca889b4a2e9626db863dba01ca'
print(api_key)



8bd0a8ca889b4a2e9626db863dba01ca


In [17]:
# Create a newsapi client

newsapi = NewsApiClient(api_key)

In [32]:
# Fetch the Bitcoin news articles

btc_everything = newsapi.get_everything(q="bitcoin AND Bitcoin AND BTC ", language="en",
    page_size=100,sort_by="relevancy"
)
print(f"Total articles about bitcoin: {btc_everything['totalResults']}")
btc_everything["articles"][0]

Total articles about bitcoin: 1425


{'source': {'id': 'mashable', 'name': 'Mashable'},
 'author': 'Stan Schroeder',
 'title': 'A massive Bitcoin event is coming next week. Here’s what you need to know.',
 'description': 'Early next week, Bitcoin is about to experience its third-ever halving, a hugely important event that could shift the balance of power within the network. Historically the event, also called the halvening, drove the price of Bitcoin up, especially in the long…',
 'url': 'https://mashable.com/article/bitcoin-halving-2020/',
 'urlToImage': 'https://mondrian.mashable.com/2020%252F05%252F08%252Fa8%252Fc07c26da4a034343ac4f3658790fbf1b.f03d7.png%252F1200x630.png?signature=arGT86ntGpF3sw7-o8Hqu_k4wLw=',
 'publishedAt': '2020-05-08T09:00:00Z',
 'content': 'Early next week, Bitcoin is about to experience its third-ever halving, a hugely important event that could shift the balance of power within the network. Historically the event, also called the halv… [+7424 chars]'}

In [31]:
# Fetch the Ethereum news articles
eth_everything = newsapi.get_everything(q='ethereum AND Ethereum AND ETH', page_size=100,
                                         language='en', sort_by="relevancy"
                                          )
print(f"Total articles about ethereum: {eth_everything['totalResults']}")
eth_everything["articles"][0]

Total articles about ethereum: 288


{'source': {'id': None, 'name': 'CoinDesk'},
 'author': 'William Foxley',
 'title': 'Schlesi Testnet Is Latest Step in Long Road Toward Eth 2.0',
 'description': "Ethereum clients are beginning to sync and validate a new Eth 2.0 testnet, Schlesi, ahead of the network's prospective launch in July.",
 'url': 'https://www.coindesk.com/ethereum-schlesi-testnet-eth-2-0',
 'urlToImage': 'https://www.coindesk.com/wp-content/uploads/2020/05/Ubahn-Schlesischestor-1200x628.jpeg',
 'publishedAt': '2020-05-07T16:47:54Z',
 'content': 'Another Eth 2.0 testnet, Schlesi, is live in what amounts to one more step in the trek toward the Proof-of-Stake (PoS) Eth 2.0, the next iteration of the Ethereum blockchain.\r\nNamed after a rail line… [+5243 chars]'}

In [36]:
# Create the Bitcoin sentiment scores DataFrame


In [2140]:
btc_sentiments = []

for article in btc_everything["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
    btc_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
btc_df = pd.DataFrame(btc_sentiments)


# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
btc_df = btc_df[cols]

btc_df.head()

,date,text,compound,positive,negative,neutral
0,2020-03-12,Bitcoin is going through a remarkably bad day....,-0.2944,0.074,0.078,0.847
1,2020-03-04,After launching two cryptocurrency-oriented sm...,0.0000,0.000,0.000,1.000
2,2020-03-04,Indias Supreme Court on Wednesday overturned c...,-0.5106,0.103,0.168,0.730
3,2020-03-19,Our robot colleague Satoshi Nakaboto writes ab...,0.5070,0.068,0.000,0.932
4,2020-03-16,Former Coinbase chief legal officer Brian Broo...,0.4939,0.085,0.000,0.915


In [2509]:
# Create the Facebook Libra sentiment scores DataFrame
eth_sentiments = []

for article in eth_everything["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        eth_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
eth_df = pd.DataFrame(eth_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
eth_df = eth_df[cols]

eth_df.head()

,date,text,compound,positive,negative,neutral
0,2020-03-12,Bitcoin is going through a remarkably bad day....,-0.2944,0.074,0.078,0.847
1,2020-03-06,If a player picks up an item in an online vide...,0.0000,0.000,0.000,1.000
2,2020-03-17,"Coinbase Card, the Visa debit card that lets y...",-0.1027,0.000,0.035,0.965
3,2020-02-27,When it comes to celebrities shilling shady in...,0.3182,0.112,0.066,0.822
4,2020-03-05,The South Korean National Assembly passed new ...,0.4019,0.068,0.000,0.932


In [2537]:
# Describe the Bitcoin Sentiment
btc_df.describe()

,compound,positive,negative,neutral
count,94.000000,94.000000,94.000000,94.000000
mean,0.124687,0.080362,0.056277,0.863394
std,0.526932,0.064738,0.067151,0.084879
min,-0.933700,0.000000,0.000000,0.616000
25%,-0.371350,0.045250,0.000000,0.821000
50%,0.307100,0.067500,0.042000,0.881000
75%,0.507000,0.108500,0.098750,0.932000
max,0.923100,0.331000,0.295000,1.000000


In [2564]:
# Describe the Ethereum Sentiment
eth_df.describe()

,compound,positive,negative,neutral
count,98.000000,98.000000,98.000000,98.000000
mean,0.239585,0.076224,0.029255,0.894520
std,0.434369,0.064692,0.044163,0.070109
min,-0.717400,0.000000,0.000000,0.637000
25%,0.000000,0.000000,0.000000,0.847000
50%,0.340000,0.068000,0.000000,0.908000
75%,0.571900,0.114500,0.047750,0.944000
max,0.895500,0.266000,0.201000,1.000000


### Questions:

Q: Which coin had the highest mean positive score?

A: Bitcoin

Q: Which coin had the highest compound score?

A: Ethereum

Q. Which coin had the highest positive score?

A: Ethereum

---

# Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word
2. Remove Punctuation
3. Remove Stopwords

In [40]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

lemmatizer = WordNetLemmatizer()

In [41]:
# Expand the default stopwords list if necessary
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [42]:
# btc_everything.get('articles')[1]
# btc_articles=  btc_everything.get('articles')[0]
# btc_articles.get("content")

TypeError: 'NoneType' object is not subscriptable

In [44]:
 # eth_everything.get('articles')[1]
# eth_articles=  eth_everything.get('articles')[0]
# eth_articles.get("content")

'Another Eth 2.0 testnet, Schlesi, is live in what amounts to one more step in the trek toward the Proof-of-Stake (PoS) Eth 2.0, the next iteration of the Ethereum blockchain.\r\nNamed after a rail line… [+5243 chars]'

In [52]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
# Complete the tokenizer function
   # Remove the stop words
    sw = set(stopwords.words('english'))
    
    # Remove the punctuation
    regex = re.compile('[^a-zA-Z ]')
    
    re_clean = regex.sub('', text)
    # Create a list of the words
    words = word_tokenize(re_clean)
    
    # Lemmatize Words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
    
    # Convert the words to lowercase
    tokens = [word.lower() for word in lem if word.lower() not in sw]
    
    return tokens


In [53]:
btc_everything['tokens'] = [tokenizer(t) for t in btc_everything['text']]
btc_everything.head()

,text,compound,positive,negative,neutral,tokens
0,"Early next week, Bitcoin is about to experienc...",0.2716,0.061,0.000,0.939,"[early, next, week, bitcoin, experience, third..."
1,Since the first Bitcoin block was generated ba...,-0.5859,0.000,0.134,0.866,"[since, first, bitcoin, block, wa, generated, ..."
2,<ul><li>The cryptocurrency market added $13 bi...,0.0772,0.066,0.058,0.877,"[ullithe, cryptocurrency, market, added, billi..."
3,Our robot colleague Satoshi Nakaboto writes ab...,0.5070,0.090,0.000,0.910,"[robot, colleague, satoshi, nakaboto, writes, ..."
4,Bitcoin BTC is nearing one of its most importa...,0.5563,0.145,0.055,0.800,"[bitcoin, btc, nearing, one, important, milest..."


In [54]:
# Create a new tokens column for bitcoin
eth_everything['tokens']=[tokenizer(word) for word in eth_everything['text']]
eth_everything.head()

KeyError: 'text'

In [4316]:
# Create a new tokens column for ethereum
eth_df.iloc[-5:,5]
print

<function print>

---

# NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [57]:
from collections import Counter
from nltk import ngrams


In [58]:
processed = tokenizer(text)
print(processed)

['gettyit', 'nothing', 'new', 'equity', 'watch', 'index', 'rise', 'towards', 'round', 'number', 'fall', 'back', 'touch', 'new', 'generation', 'crypto', 'hodlers', 'new', 'experiencethere', 'always', 'char']


In [59]:
word_counts = Counter(processed)
print(dict(word_counts))


{'gettyit': 1, 'nothing': 1, 'new': 3, 'equity': 1, 'watch': 1, 'index': 1, 'rise': 1, 'towards': 1, 'round': 1, 'number': 1, 'fall': 1, 'back': 1, 'touch': 1, 'generation': 1, 'crypto': 1, 'hodlers': 1, 'experiencethere': 1, 'always': 1, 'char': 1}


In [60]:
# Generate the Bitcoin N-grams where N=2
def bigram_counter(text):
    
    big_string = ''.join(text)
    tokens = tokenizer(big_string)
    bigrams = ngrams(tokens, n=2)
    
    return Counter(bigrams).most_common(10)



# Generate Bitcoin N-grams
bigram_counter(btc_everything['text'])

[(('satoshi', 'nakaboto'), 28),
 (('bitcoin', 'btc'), 21),
 (('charsour', 'robot'), 14),
 (('robot', 'colleague'), 14),
 (('colleague', 'satoshi'), 14),
 (('nakaboto', 'writes'), 14),
 (('writes', 'bitcoin'), 14),
 (('btc', 'every'), 14),
 (('every', 'fucking'), 14),
 (('fucking', 'daywelcome'), 14)]

In [65]:
# Generate the Ethereum N-grams where N=2
bigram_counter(eth_everything['text'])

KeyError: 'text'

In [66]:
# Use the token_count function to generate the top 10 words from each coin
def token_count(tokens, N=10):
    print(dict(bigram_counts.most_common(10)))
    return Counter(tokens).most_common(10)

In [79]:
# Get the top 10 words for Bitcoin
btc_everything = []
for row in btc_everything['Tokens']:
    for token in row:
       btc_everything.append(token)
        
token_count(btc_everything)

TypeError: list indices must be integers or slices, not str

In [80]:
# Get the top 10 words for Ethereum
eth_everything = []

for row in eth_everything['tokens']:
    for token in row:
        eth_everything.append(token)
        
token_count(eth_everything)

TypeError: list indices must be integers or slices, not str

# Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [73]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [76]:
# Generate the Bitcoin word cloud
bitcoin_tokens = ','.join(btc_everything)
wc_b = WordCloud().generate(bitcoin_tokens)
plt.title('Bitcoin Word Cloud', fontsize=40)
plt.imshow(wc_b)

ValueError: We need at least 1 word to plot a word cloud, got 0.

In [81]:
# Generate the Ethereum word cloud
ethereum_tokens = ','.join(eth_everything)
wc_e = WordCloud().generate(eth_everything)
plt.title('Ethereum Word Cloud', fontsize=40)
plt.imshow(wc_e)

TypeError: expected string or bytes-like object

# Named Entity Recognition

In this section, you will build a named entity recognition model for both coins and visualize the tags using SpaCy.

In [4723]:
import spacy
from spacy import displacy

In [1]:
# Optional - download a language model for SpaCy
#!python -m spacy download en_core_web_sm

In [4744]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

## Bitcoin NER

In [2]:
# Concatenate all of the bitcoin text together
bitcoin_string = ''
for row in bitcoin_sentiments['Text']:
    bitcoin_string = bitcoin_string + row
    
bitcoin_string[:4000]

NameError: name 'bitcoin_sentiments' is not defined

In [4746]:
# Run the NER processor on all of the text
bit_doc=nlp(bitcoin_string[:4000])

# Add a title to the document
bit_doc.user_data["title"] = "Bitcoin NER"

In [4747]:
# Render the visualization
displacy.render(bit_doc, style='ent', jupyter=True)

In [4748]:
# List all Entities
bit_entities = [ent.text for ent in bit_doc.ents]
bit_entities[:200]

---

## Ethereum NER

In [83]:
# Concatenate all of the bitcoin text together
ethereum_string = ''
for row in ethereum_sentiments['Text']:
    ethereum_string = ethereum_string + row
    
ethereum_string[:4000]

NameError: name 'ethereum_sentiment' is not defined

In [4750]:

# Run the NER processor on all of the text
eth_doc = nlp(ethereum_string[:4000])

# Add a title to the document
eth_title = eth_doc.user_data["title"] = "Ethereum NER"

In [4751]:
# Render the visualization
displacy.render(eth_doc, style='ent')

In [4752]:
# List all Entities
eth_entities = [ent.text for ent in eth_doc.ents]
eth_entities[:200]